In [68]:
import pandas as pd
import numpy as np
import torch

import transformers
import pickle
import os

import nlpsig
import nlpsig_networks

from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from nlpsig.classification_utils import split_dataset
from nlpsig_networks.pytorch_utils import training_pytorch, testing_pytorch, set_seed
from nlpsig_networks.ffn import FeedforwardNeuralNetModel
from nlpsig_networks.deepsignet import StackedDeepSigNet
from nlpsig_networks.focal_loss import FocalLoss, ClassBalanced_FocalLoss
from sklearn import metrics

from tqdm.notebook import tqdm

seed = 2023

## AnnoMI

In [2]:
anno_mi = pd.read_csv("AnnoMI-full.csv")
anno_mi["datetime"] = pd.to_datetime(anno_mi["timestamp"])
anno_mi = anno_mi.drop(columns=["video_title", "video_url"])
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-05-05 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-05-05 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-05-05 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-05-05 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-05-05 00:00:34


In [3]:
len(anno_mi)

13551

In [4]:
anno_mi["client_talk_type"].value_counts() / anno_mi["interlocutor"].value_counts()["client"]

neutral    0.627063
change     0.248030
sustain    0.124907
Name: client_talk_type, dtype: float64

In [5]:
anno_mi["interlocutor"].value_counts()

therapist    6826
client       6725
Name: interlocutor, dtype: int64

In [6]:
anno_mi["topic"].value_counts()

reducing alcohol consumption                                                          2326
more exercise / increasing activity                                                   2034
reducing recidivism                                                                   1303
reducing drug use                                                                     1104
diabetes management                                                                    948
smoking cessation                                                                      923
smoking cessation                                                                      541
taking medicine / following medical procedure                                          448
asthma management                                                                      431
avoiding DOI                                                                           394
changing approach to disease                                                           315

In [7]:
len(anno_mi["transcript_id"].unique())

133

## Only considering client for now...

In [8]:
client_index = [isinstance(x, str) for x in anno_mi["client_talk_type"]]
sum(client_index)

6725

In [9]:
y_data = anno_mi["client_talk_type"][client_index]
y_data.shape

(6725,)

In [10]:
y_data[0:20]

1     neutral
3     neutral
5     neutral
7     neutral
9     neutral
11    neutral
13    neutral
15    neutral
17    neutral
19    neutral
21    neutral
23    neutral
25    neutral
27    neutral
29    neutral
31    neutral
33    neutral
35     change
37     change
39     change
Name: client_talk_type, dtype: object

In [11]:
label_to_id = {y_data.unique()[i]: i for i in range(len(y_data.unique()))}
id_to_label = {v: k for k, v in label_to_id.items()}

In [12]:
label_to_id

{'neutral': 0, 'change': 1, 'sustain': 2}

In [13]:
id_to_label

{0: 'neutral', 1: 'change', 2: 'sustain'}

In [14]:
y_data = [label_to_id[x] for x in y_data]
y_data[0:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

## Obtaining SBERT Embeddings

We can use the `SentenceEncoder` class within `nlpsig` to obtain sentence embeddings from a model. This class uses the [`sentence-transformer`](https://www.sbert.net/docs/package_reference/SentenceTransformer.html) package and here, we have use the pre-trained `all-mpnet-base-v2` model by passing this name as a string to the class - alternative models can be found [here](https://www.sbert.net/docs/pretrained_models.html).

We can pass these into the constructor of the class to initialise our text encoder as follows:

In [15]:
# sbert_768_embeddings = np.load("anno_mi_sentence_embeddings_768.npy")

In [16]:
# initialise the Text Encoder
sbert_model_768 = "all-mpnet-base-v2"
text_encoder_sbert_768 = nlpsig.SentenceEncoder(df=anno_mi,
                                                feature_name="utterance_text",
                                                model_name=sbert_model_768)
text_encoder_sbert_768.load_pretrained_model()

The class has a `.encode_sentence_transformer()` method which first loads in the model (using the `model_name` and `model_args` attributes) and then obtains an embedding for each sentence. These sentence embeddings are then stored in the `embeddings_sentence` attribute of the object.

In [17]:
text_encoder_sbert_768.obtain_embeddings()
sbert_768_embeddings = text_encoder_sbert_768.sentence_embeddings

[INFO] number of sentences to encode: 13551


Batches:   0%|          | 0/212 [00:00<?, ?it/s]

In [18]:
np.save("anno_mi_sentence_embeddings_768", sbert_768_embeddings)

## SBERT with 384 dimension vectors

In [16]:
# sbert_384_embeddings = np.load("anno_mi_sentence_embeddings_384.npy")

In [20]:
# initialise the Text Encoder
sbert_model_384 = "all-MiniLM-L12-v2"
text_encoder_sbert_384 = nlpsig.SentenceEncoder(df=anno_mi,
                                                feature_name="utterance_text",
                                                model_name=sbert_model_384)
text_encoder_sbert_384.load_pretrained_model()

In [21]:
text_encoder_sbert_384.obtain_embeddings()
sbert_384_embeddings = text_encoder_sbert_384.sentence_embeddings

[INFO] number of sentences to encode: 13551


Batches:   0%|          | 0/212 [00:00<?, ?it/s]

In [22]:
np.save("anno_mi_sentence_embeddings_384", sbert_384_embeddings)

## Pretrained BERT and pooling

In [17]:
# pooled_mean_pretrained = np.load("anno_mi_pretrained_BERT_mean.npy")
# pooled_max_pretrained = np.load("anno_mi_pretrained_BERT_max.npy")
# pooled_sum_pretrained = np.load("anno_mi_pretrained_BERT_sum.npy")
# pooled_cls_pretrained = np.load("anno_mi_pretrained_BERT_cls.npy")

In [24]:
bert_model = "bert-base-uncased"

In [25]:
text_encoder_pretrained_BERT = nlpsig.TextEncoder(df=anno_mi,
                                                  feature_name="utterance_text",
                                                  model_name=bert_model)
text_encoder_pretrained_BERT.load_pretrained_model()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
text_encoder_pretrained_BERT.tokenize_text(skip_special_tokens=False)

[INFO] Setting return_special_tokens_mask=True
[INFO] Tokenizing the dataset...


Map:   0%|          | 0/13551 [00:00<?, ? examples/s]

[INFO] Saving the tokenized text for each sentence into `.df['tokens']`...


Map:   0%|          | 0/13551 [00:00<?, ? examples/s]

[INFO] Creating tokenized dataframe and setting in `.tokenized_df` attribute...
[INFO] Note: 'text_id' is the column name for denoting the corresponding text id


Dataset({
    features: ['mi_quality', 'transcript_id', 'topic', 'utterance_id', 'interlocutor', 'timestamp', 'utterance_text', 'annotator_id', 'therapist_input_exists', 'therapist_input_subtype', 'reflection_exists', 'reflection_subtype', 'question_exists', 'question_subtype', 'main_therapist_behaviour', 'client_talk_type', 'datetime', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask', 'tokens'],
    num_rows: 13551
})

In [27]:
token_embeddings_pretrained = text_encoder_pretrained_BERT.obtain_embeddings(method="hidden_layer")

  0%|          | 0/136 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [28]:
pooled_mean_pretrained = text_encoder_pretrained_BERT.pool_token_embeddings()
pooled_max_pretrained = text_encoder_pretrained_BERT.pool_token_embeddings(method="max")
pooled_sum_pretrained = text_encoder_pretrained_BERT.pool_token_embeddings(method="sum") 
pooled_cls_pretrained = text_encoder_pretrained_BERT.pool_token_embeddings(method="cls")

  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/13551 [00:00<?, ?it/s]

In [29]:
pooled_mean_pretrained.shape

(13551, 768)

In [30]:
pooled_max_pretrained.shape

(13551, 768)

In [31]:
pooled_sum_pretrained.shape

(13551, 768)

In [32]:
pooled_cls_pretrained.shape

(13551, 768)

In [33]:
np.save("anno_mi_pretrained_BERT_mean", pooled_mean_pretrained)
np.save("anno_mi_pretrained_BERT_max", pooled_max_pretrained)
np.save("anno_mi_pretrained_BERT_sum", pooled_sum_pretrained)
np.save("anno_mi_pretrained_BERT_cls", pooled_cls_pretrained)

## Fine-tuning BERT and pooling

### (Ignoring this part for now while, but will run this on GPU cluster soon...)

In [34]:
# pooled_mean = np.load("anno_mi_BERT_mean.npy")
# pooled_max = np.load("anno_mi_BERT_max.npy")
# pooled_sum = np.load("anno_mi_BERT_sum.npy")
# pooled_cls = np.load("anno_mi_BERT_cls.npy")

In [35]:
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorWithPadding,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline,
)

model = AutoModelForMaskedLM.from_pretrained(bert_model)
tokenizer = AutoTokenizer.from_pretrained(bert_model)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
text_encoder_BERT = nlpsig.TextEncoder(df=anno_mi,
                                       feature_name="utterance_text",
                                       model=model,
                                       tokenizer=tokenizer,
                                       data_collator=data_collator)

In [37]:
text_encoder_BERT.tokenize_text(skip_special_tokens=False)

[INFO] Setting return_special_tokens_mask=True
[INFO] Tokenizing the dataset...


Map:   0%|          | 0/13551 [00:00<?, ? examples/s]

[INFO] Saving the tokenized text for each sentence into `.df['tokens']`...


Map:   0%|          | 0/13551 [00:00<?, ? examples/s]

[INFO] Creating tokenized dataframe and setting in `.tokenized_df` attribute...
[INFO] Note: 'text_id' is the column name for denoting the corresponding text id


Dataset({
    features: ['mi_quality', 'transcript_id', 'topic', 'utterance_id', 'interlocutor', 'timestamp', 'utterance_text', 'annotator_id', 'therapist_input_exists', 'therapist_input_subtype', 'reflection_exists', 'reflection_subtype', 'question_exists', 'question_subtype', 'main_therapist_behaviour', 'client_talk_type', 'datetime', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 13551
})

### Training the model

In [38]:
# set up data_collator for language modelling (has dynamic padding)
data_collator_for_LM = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                       mlm=True,
                                                       mlm_probability=0.15)

In [39]:
text_encoder_BERT.split_dataset(seed=seed)

[INFO] Splitting up dataset into train / validation / test sets, and saving to `.dataset_split`.


DatasetDict({
    train: Dataset({
        features: ['mi_quality', 'transcript_id', 'topic', 'utterance_id', 'interlocutor', 'timestamp', 'utterance_text', 'annotator_id', 'therapist_input_exists', 'therapist_input_subtype', 'reflection_exists', 'reflection_subtype', 'question_exists', 'question_subtype', 'main_therapist_behaviour', 'client_talk_type', 'datetime', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 10840
    })
    test: Dataset({
        features: ['mi_quality', 'transcript_id', 'topic', 'utterance_id', 'interlocutor', 'timestamp', 'utterance_text', 'annotator_id', 'therapist_input_exists', 'therapist_input_subtype', 'reflection_exists', 'reflection_subtype', 'question_exists', 'question_subtype', 'main_therapist_behaviour', 'client_talk_type', 'datetime', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 1356
    })
    validation: Dataset({
        features: ['mi_qua

In [40]:
type(text_encoder_BERT.dataset_split)

datasets.dataset_dict.DatasetDict

In [41]:
model_name = "bert-base-uncased-anno-mi"
text_encoder_BERT.set_up_training_args(output_dir=model_name,
                                  num_train_epochs=600,
                                  per_device_train_batch_size=128,
                                  disable_tqdm=False,
                                  save_strategy="steps",
                                  save_steps=10000,
                                  seed=seed)

[INFO] Setting up TrainingArguments object and saving to `.training_args`.


TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ign

In [42]:
type(text_encoder_BERT.training_args)

transformers.training_args.TrainingArguments

In [43]:
text_encoder_BERT.set_up_trainer(data_collator=data_collator_for_LM)

[INFO] Setting up Trainer object, and saving to `.trainer`.


In [44]:
type(text_encoder_BERT.trainer)

transformers.trainer.Trainer

In [45]:
torch.cuda.is_available()

False

In [46]:
torch.cuda.device_count()

0

In [47]:
# set to only report errors to avoid excessing logging
transformers.utils.logging.set_verbosity(40)

In [ ]:
text_encoder_BERT.fit_transformer_with_trainer_api()

[INFO] Training model with 109514298 parameters...


/Users/rchan/opt/miniconda3/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
text_encoder_BERT.trainer.save_model(model_name)

### Evaluating model on masked language modelling task

In [ ]:
text_encoder_BERT.tokenizer.special_tokens_map

In [ ]:
def compute_masked_character_accuracy(fill_mask, words):
    was_correct = []
    print(f"Evaluating with {len(words)} words")
    for word in tqdm(words):
        masked_strings = [word[:i] + '<mask>' + word[i+1:] for i in range(len(word))]
        predictions = [fill_mask(word)[0]['sequence'] for word in masked_strings]
        was_correct += [pred == word for pred in predictions]
    
    acc = np.sum(was_correct) / len(was_correct)
    print(f"Accuracy: {acc}")
    return acc

In [ ]:
fill_mask = pipeline("fill-mask",
                     model=model_name,
                     tokenizer=model_name)

compute_masked_character_accuracy(fill_mask, text_encoder_BERT.dataset_split["test"]["word"])

### Obtain embeddings from model

In [ ]:
# setting the model to CPU (might not be always necessary to run this)
text_encoder_BERT.model.to('cpu')
token_embeddings = text_encoder_BERT.obtain_embeddings(method="hidden_layer")

In [ ]:
token_embeddings.shape

In [ ]:
pooled_mean = text_encoder_BERT.pool_token_embeddings()
pooled_max = text_encoder_BERT.pool_token_embeddings(method="max")
pooled_sum = text_encoder_BERT.pool_token_embeddings(method="sum")
pooled_cls = text_encoder_BERT.pool_token_embeddings(method="cls")

In [ ]:
pooled_mean.shape

In [ ]:
pooled_max.shape

In [ ]:
pooled_sum.shape

In [ ]:
pooled_cls.shape

In [ ]:
np.save("anno_mi_BERT_mean", pooled_mean)
np.save("anno_mi_BERT_max", pooled_max)
np.save("anno_mi_BERT_sum", pooled_sum)
np.save("anno_mi_BERT_cls", pooled_cls)

# Baseline 1: FFN baseline

Using the embeddings for the sentences directly in a FFN.

Below is a function that takes in some inputs x_data, y_data and fits a FFN. Will do early stopping if the F1 score continually does not improve (patience is a bit high since we actually count if the F1 isn't changed in an epoch, so we make it a bit higher to allow it to stay at the same F1 performance for a few epochs basically).

In [154]:
def implement_ffn(x_data,
                  y_data,
                  hidden_dim,
                  learning_rate,
                  loss,
                  gamma=0):
    # set seed
    set_seed(seed)
    
    # initialise FFN
    ffn_model = FeedforwardNeuralNetModel(input_dim=x_data.shape[1],
                                          hidden_dim=hidden_dim,
                                          output_dim=len(label_to_id),
                                          dropout_rate=0.1)
    
    # split dataset
    train, valid, test = split_dataset(x_data=torch.tensor(x_data).float(),
                                       y_data=torch.tensor(y_data),
                                       train_size=0.8,
                                       valid_size=0.5,
                                       shuffle=True,
                                       as_DataLoader=True,
                                       seed=seed)

    # define loss
    if loss == "focal":
        criterion = FocalLoss(gamma = gamma)
    elif loss == "cross_entropy":
        criterion = torch.nn.CrossEntropyLoss()

    # define optimizer
    optimizer = torch.optim.Adam(ffn_model.parameters(), lr=learning_rate)
    # define scheduler for adjusting the learning rate
    scheduler = ReduceLROnPlateau(optimizer, 'min')
    # scheduler = StepLR(optimizer, step_size = 4, gamma = 0.5)
    # scheduler = None
    
    ffn_model = training_pytorch(model=ffn_model,
                                 train_loader=train,
                                 criterion=criterion,
                                 optimizer=optimizer,
                                 num_epochs=10000,
                                 scheduler=scheduler,
                                 valid_loader=valid,
                                 seed=seed,
                                 early_stopping=True,
                                 early_stopping_metric="f1",
                                 patience=100,
                                 verbose=True,
                                 verbose_epoch=100)

    pred, label = testing_pytorch(ffn_model, test, criterion)
    print(f"proportion of labels in prediction: {[sum(pred==i)/len(pred) for i in label_to_id.values()]}")
    print(f"proportion of labels in data: {[sum(label==i)/len(label) for i in label_to_id.values()]}")
    
    f1_scores = metrics.f1_score(label, pred, average=None)
    print(f"- f1: {f1_scores}")
    print(f"- f1 (average): {sum(f1_scores)/len(f1_scores)}")
    print(f"- accuracy: {sum(pred==label)/len(pred)}")
    
    return ffn_model

Going to try out some variations (1 hidden layer, 2 hidden layers and 3 hidden layers - all of size 100)

In [155]:
hidden_dim_trials = [100, [100,100], [100,100,100]]
learning_rate = 2e-5
loss = "cross_entropy"

## SBERT 768

In [156]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=sbert_768_embeddings[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0885392427444458
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0795083045959473
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0757082592357288 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.5589850544929504
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.9038074016571045
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.6840004433285106 || Accuracy: 0.6953937411308289 || F1-score: 0.532593092563333
Epoch: 201/10000 || Item: 0/85 || Loss: 0.7341877222061157
--------------------------------------------------
##### Epoch: 201/10000 || Loss: 1.368276834487915
--------------------------------------------------
Validation || Epoch: 201 || Loss: 0.6921473199670966 || Accuracy: 0.7013372778892517 || F1-score: 0.5449254096356076
Early stopping at ep

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.116580843925476
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.118660807609558
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.1014031171798706 || Accuracy: 0.12184249609708786 || F1-score: 0.07240618101545253
Epoch: 101/10000 || Item: 0/85 || Loss: 0.7407606244087219
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4813379645347595
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.685649649663405 || Accuracy: 0.6939078569412231 || F1-score: 0.521815389563948
Epoch: 201/10000 || Item: 0/85 || Loss: 0.7610892653465271
--------------------------------------------------
##### Epoch: 201/10000 || Loss: 1.2291091680526733
--------------------------------------------------
Validation || Epoch: 201 || Loss: 0.6832592649893328 || Accuracy: 0.7028231620788574 || F1-score: 0.5480318366095817
Early stopping at epo

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0993850231170654
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.084618330001831
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0871503244746814 || Accuracy: 0.37147101759910583 || F1-score: 0.2602133749674733
Epoch: 101/10000 || Item: 0/85 || Loss: 0.6977670192718506
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.1788264811038971
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7017621939832513 || Accuracy: 0.7013372778892517 || F1-score: 0.4817348905283425
Epoch: 201/10000 || Item: 0/85 || Loss: 0.6531517505645752
--------------------------------------------------
##### Epoch: 201/10000 || Loss: 0.25363898277282715
--------------------------------------------------
Validation || Epoch: 201 || Loss: 0.6932478276166049 || Accuracy: 0.7028231620788574 || F1-score: 0.5252829038733572
Early stopping at 

## SBERT 384

In [95]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=sbert_384_embeddings[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0692368745803833
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0597608089447021
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0575233806263318 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.5714445114135742
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.993977427482605
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7121773145415566 || Accuracy: 0.6775631308555603 || F1-score: 0.4852881287212332
Early stopping at epoch 193!
Accuracy on dataset of size 672: 69.49404907226562 %.
Average loss: 0.7372136386958036
proportion of labels in prediction: [tensor(0.7693), tensor(0.1786), tensor(0.0521)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.80982906 0.51111111 0.27536232]
- f1 (average): 0.532100829

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.1142284870147705
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0898610353469849
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.1067617481405085 || Accuracy: 0.25557205080986023 || F1-score: 0.13570019723865875
Epoch: 101/10000 || Item: 0/85 || Loss: 0.7563970685005188
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.24487124383449554
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7194767540151422 || Accuracy: 0.6864784359931946 || F1-score: 0.48013654513009985
Epoch: 201/10000 || Item: 0/85 || Loss: 0.7156612277030945
--------------------------------------------------
##### Epoch: 201/10000 || Loss: 1.055163860321045
--------------------------------------------------
Validation || Epoch: 201 || Loss: 0.7143634991212324 || Accuracy: 0.6924219727516174 || F1-score: 0.5152181344117231
Early stopping a

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0598180294036865
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0403138399124146
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.051291660829024 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8030747175216675
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.3111073076725006
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7629680037498474 || Accuracy: 0.6627042889595032 || F1-score: 0.41140123154470193
Early stopping at epoch 174!
Accuracy on dataset of size 672: 66.51786041259766 %.
Average loss: 0.7649482705376365
proportion of labels in prediction: [tensor(0.7321), tensor(0.2679), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.82107574 0.44242424 0.        ]
- f1 (average): 0.421166661122

## Pretrained BERT

### Mean pooled

In [96]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_mean_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0839786529541016
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0357847213745117
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0042308189652183 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.5391806364059448
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.6543185114860535
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7084653865207325 || Accuracy: 0.6879643201828003 || F1-score: 0.5493394831286408
Early stopping at epoch 186!
Accuracy on dataset of size 672: 69.04762268066406 %.
Average loss: 0.7226935570890253
proportion of labels in prediction: [tensor(0.7396), tensor(0.2009), tensor(0.0595)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.80567686 0.51929825 0.29370629]
- f1 (average): 0.53956046

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.1131868362426758
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0881928205490112
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.070029453797774 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.7127247452735901
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4683959186077118
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.6954461498693987 || Accuracy: 0.7028231620788574 || F1-score: 0.5731369952368576
Early stopping at epoch 194!
Accuracy on dataset of size 672: 68.75 %.
Average loss: 0.7084680687297474
proportion of labels in prediction: [tensor(0.7277), tensor(0.2024), tensor(0.0699)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.80396476 0.51748252 0.30666667]
- f1 (average): 0.5427046472861451
- ac

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.098788857460022
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0711716413497925
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0712726332924583 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.6781942248344421
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.12910042703151703
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.6977984309196472 || Accuracy: 0.6939078569412231 || F1-score: 0.5608619843873767
Early stopping at epoch 180!
Accuracy on dataset of size 672: 69.19642639160156 %.
Average loss: 0.7155460823665966
proportion of labels in prediction: [tensor(0.7277), tensor(0.1979), tensor(0.0744)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.81057269 0.50883392 0.32679739]
- f1 (average): 0.54873466

### Max pooled

In [97]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_max_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0957067012786865
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0682802200317383
--------------------------------------------------
Validation || Epoch: 1 || Loss: 0.969028727574782 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.571304440498352
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.8043275475502014
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7314741611480713 || Accuracy: 0.6716195940971375 || F1-score: 0.506992275294162
Early stopping at epoch 178!
Accuracy on dataset of size 672: 68.00595092773438 %.
Average loss: 0.7330376018177379
proportion of labels in prediction: [tensor(0.7619), tensor(0.2068), tensor(0.0312)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.80558539 0.47750865 0.20967742]
- f1 (average): 0.49759048730

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.1076050996780396
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0695494413375854
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0264393470504067 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8345953226089478
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.3302067220211029
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.721508036960255 || Accuracy: 0.6775631308555603 || F1-score: 0.5272523330049675
Early stopping at epoch 193!
Accuracy on dataset of size 672: 69.3452377319336 %.
Average loss: 0.7208894978870045
proportion of labels in prediction: [tensor(0.7500), tensor(0.2009), tensor(0.0491)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.80823402 0.5122807  0.29411765]
- f1 (average): 0.5382107894

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0997354984283447
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0507149696350098
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0337201248515735 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.7388246655464172
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.18091349303722382
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7373271042650397 || Accuracy: 0.6760772466659546 || F1-score: 0.5229165624322477
Epoch: 201/10000 || Item: 0/85 || Loss: 0.7790274620056152
--------------------------------------------------
##### Epoch: 201/10000 || Loss: 0.18720248341560364
--------------------------------------------------
Validation || Epoch: 201 || Loss: 0.7315309806303545 || Accuracy: 0.6805348992347717 || F1-score: 0.5289228692454498
Early stopping a

### Sum pooled

In [98]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_sum_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 2.0271031856536865
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0212513208389282
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0024839531291614 || Accuracy: 0.6225854158401489 || F1-score: 0.3232245025425277
Epoch: 101/10000 || Item: 0/85 || Loss: 0.4351794719696045
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.2922021746635437
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.72612741318616 || Accuracy: 0.6909360885620117 || F1-score: 0.565352009350549
Early stopping at epoch 136!
Accuracy on dataset of size 672: 70.68452453613281 %.
Average loss: 0.7379533269188621
proportion of labels in prediction: [tensor(0.7485), tensor(0.1786), tensor(0.0729)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.80477223 0.54074074 0.40789474]
- f1 (average): 0.584469237285

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.114878535270691
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0557730197906494
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.054190386425365 || Accuracy: 0.6151559948921204 || F1-score: 0.34358449687048626
Epoch: 101/10000 || Item: 0/85 || Loss: 0.736968994140625
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.3847038149833679
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7197055708278309 || Accuracy: 0.7028231620788574 || F1-score: 0.5803708684031409
Early stopping at epoch 151!
Accuracy on dataset of size 672: 71.57737731933594 %.
Average loss: 0.7132322219285098
proportion of labels in prediction: [tensor(0.7426), tensor(0.1860), tensor(0.0714)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.81263617 0.56       0.41059603]
- f1 (average): 0.59441073068

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.111946940422058
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.1200898885726929
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0467318513176658 || Accuracy: 0.6344724893569946 || F1-score: 0.293682004525378
Epoch: 101/10000 || Item: 0/85 || Loss: 0.6803026795387268
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.07645384967327118
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7378906770185991 || Accuracy: 0.7072808146476746 || F1-score: 0.5981767162832544
Early stopping at epoch 139!
Accuracy on dataset of size 672: 71.13095092773438 %.
Average loss: 0.736527681350708
proportion of labels in prediction: [tensor(0.7232), tensor(0.1860), tensor(0.0908)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.81104972 0.53818182 0.45121951]
- f1 (average): 0.60015035137

### CLS

In [99]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_cls_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.072586178779602
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.054619312286377
--------------------------------------------------
Validation || Epoch: 1 || Loss: 0.9783457517623901 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.5552260279655457
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.6392856240272522
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7163692333481528 || Accuracy: 0.6849925518035889 || F1-score: 0.5467744526948758
Early stopping at epoch 186!
Accuracy on dataset of size 672: 71.13095092773438 %.
Average loss: 0.7055106813257391
proportion of labels in prediction: [tensor(0.7515), tensor(0.1801), tensor(0.0685)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.81818182 0.51660517 0.40268456]
- f1 (average): 0.5791571826

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.1158097982406616
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0941648483276367
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0671783143823796 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8176752924919128
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4519164562225342
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7216152494603937 || Accuracy: 0.6805348992347717 || F1-score: 0.5284876279863481
Early stopping at epoch 149!
Accuracy on dataset of size 672: 69.3452377319336 %.
Average loss: 0.7150283401662653
proportion of labels in prediction: [tensor(0.7560), tensor(0.1696), tensor(0.0744)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.80690399 0.46969697 0.39215686]
- f1 (average): 0.556252607

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.099726676940918
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0698367357254028
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.062767429785295 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.7204791903495789
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.13758376240730286
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.7134658856825395 || Accuracy: 0.689450204372406 || F1-score: 0.56131034880422
Early stopping at epoch 182!
Accuracy on dataset of size 672: 70.68452453613281 %.
Average loss: 0.6856141903183677
proportion of labels in prediction: [tensor(0.7336), tensor(0.1830), tensor(0.0833)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.81578947 0.51282051 0.41509434]
- f1 (average): 0.581234775375

## Fine-tuned BERT

### Mean pooled

In [ ]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_mean_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=1e-5,
                  loss="cross_entropy")

### Max pooled

In [ ]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_max_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

### Sum pooled

In [ ]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_sum_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

### CLS

In [ ]:
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=pooled_sum_pretrained[client_index],
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

# Baseline 2: Averaging history and use FFN

Here, we will use `nlpsig` to construct some paths of embeddings which we will average and use those in a FFN.

First, we define the arguments for how we want to construct our path. As we're going to just do a simple average of embeddings, I'll set zero padding as false, and construct the path by looking at the last `k` posts.

In [54]:
path_specifics = {"pad_by": "history",
                  "zero_padding": False,
                  "method": "k_last",
                  "k": 5,
                  "time_feature": None,
                  "embeddings": "full",
                  "include_current_embedding": True}

In [161]:
def obtain_mean_history(embeddings, path_specifics):
    paths = nlpsig.PrepareData(anno_mi,
                               id_column="transcript_id",
                               label_column="client_talk_type",
                               embeddings=embeddings)
    path = paths.pad(**path_specifics)
    # remove last two columns (which contains the id and the label)
    path = path[client_index][:,:,:-2]
    # average in the first dimension
    return path.mean(1).astype("float")

## SBERT 768

In [169]:
path_history = obtain_mean_history(sbert_768_embeddings, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.088150978088379
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0829229354858398
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.075862082568082 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.831689178943634
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.9813627004623413
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8894803578203375 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9005365100773898
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- ac

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.1165460348129272
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.1194690465927124
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.1018761504780163 || Accuracy: 0.12184249609708786 || F1-score: 0.07240618101545253
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9759571552276611
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4108854830265045
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8946103345264088 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 102!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9107744693756104
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0994921922683716
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0858962535858154
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0874490196054631 || Accuracy: 0.2615155875682831 || F1-score: 0.14264979681024412
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9241882562637329
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.6933719515800476
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8937450593168085 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 102!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9194471185857599
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
-

## SBERT 384

In [170]:
path_history = obtain_mean_history(sbert_384_embeddings, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0689338445663452
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.059541940689087
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.05829148942774 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8537594079971313
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 1.0656259059906006
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.896614210172133 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.912129510532726
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- accu

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.114090919494629
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0903785228729248
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.106756405396895 || Accuracy: 0.25557205080986023 || F1-score: 0.13570019723865875
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9462771415710449
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4005976617336273
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8934355920011346 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 103!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9252154231071472
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- 

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0598030090332031
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0398383140563965
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0513845031911677 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9368603229522705
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.695285439491272
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.893760393966328 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9282568964091215
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- a

## Pretrained BERT

### Mean pooled

In [171]:
path_history = obtain_mean_history(pooled_mean_pretrained, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0827751159667969
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.067988395690918
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.00626910274679 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8487571477890015
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 1.1112945079803467
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.886727511882782 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9075098742138256
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- acc

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.1143455505371094
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0963679552078247
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0698934359983965 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.976948618888855
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4426864683628082
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8920709815892306 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9157748547467318
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- 

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0997600555419922
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0753464698791504
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0727270191366023 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9181236028671265
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.721269428730011
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8867454582994635 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9253213839097456
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- 

### Max pooled

In [172]:
path_history = obtain_mean_history(pooled_max_pretrained, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0947773456573486
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.159409999847412
--------------------------------------------------
Validation || Epoch: 1 || Loss: 0.9376135522669012 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8620471954345703
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 1.1281884908676147
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8903347687287764 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9101777943697843
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- 

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.1085178852081299
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0852488279342651
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0026371316476301 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9753608107566833
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4552372694015503
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8919325145808134 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.916386983611367
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- 

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.099276065826416
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0503199100494385
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0202285701578313 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.926359236240387
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.7107961177825928
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8936019160530784 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9287005175243724
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- a

### Sum pooled

In [173]:
path_history = obtain_mean_history(pooled_sum_pretrained, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.5255906581878662
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.2541639804840088
--------------------------------------------------
Validation || Epoch: 1 || Loss: 0.9284746050834656 || Accuracy: 0.6240713000297546 || F1-score: 0.26351130351130353
Epoch: 101/10000 || Item: 0/85 || Loss: 0.5938670039176941
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.7251362800598145
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8142873265526511 || Accuracy: 0.6493313312530518 || F1-score: 0.41329564888064
Early stopping at epoch 171!
Accuracy on dataset of size 672: 65.32737731933594 %.
Average loss: 0.8633222579956055
proportion of labels in prediction: [tensor(0.8795), tensor(0.1042), tensor(0.0164)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.78019802 0.31818182 0.1754386 ]
- f1 (average): 0.4246061448

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0804401636123657
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.2311005592346191
--------------------------------------------------
Validation || Epoch: 1 || Loss: 0.953440628268502 || Accuracy: 0.6210995316505432 || F1-score: 0.25542315918117936
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8102289438247681
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.4154810905456543
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8209891265088861 || Accuracy: 0.6419019103050232 || F1-score: 0.4145659757894998
Early stopping at epoch 184!
Accuracy on dataset of size 672: 65.77381134033203 %.
Average loss: 0.8609610958532854
proportion of labels in prediction: [tensor(0.8557), tensor(0.1220), tensor(0.0223)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.7806841  0.36206897 0.20338983]
- f1 (average): 0.448714300

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0874210596084595
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.1935654878616333
--------------------------------------------------
Validation || Epoch: 1 || Loss: 0.9624836119738492 || Accuracy: 0.6210995316505432 || F1-score: 0.25542315918117936
Epoch: 101/10000 || Item: 0/85 || Loss: 0.7057756781578064
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.6206573247909546
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8404642180962996 || Accuracy: 0.6389301419258118 || F1-score: 0.44752010597029046
Early stopping at epoch 172!
Accuracy on dataset of size 672: 64.73213958740234 %.
Average loss: 0.8940324891697277
proportion of labels in prediction: [tensor(0.8259), tensor(0.1414), tensor(0.0327)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76796715 0.3755102  0.24      ]
- f1 (average): 0.4611591

### CLS

In [174]:
path_history = obtain_mean_history(pooled_cls_pretrained, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]


********** hidden_dim: 100


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0739248991012573
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0877466201782227
--------------------------------------------------
Validation || Epoch: 1 || Loss: 0.9801731163805182 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.8488182425498962
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 1.0061792135238647
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8912230188196356 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9096886840733615
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
-

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.116005539894104
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0968785285949707
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0649296695535833 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9889340400695801
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.43216484785079956
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8960904695770957 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9185672185637734
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
-

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.100395917892456
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0688506364822388
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.0630166205492886 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Epoch: 101/10000 || Item: 0/85 || Loss: 0.9245232343673706
--------------------------------------------------
##### Epoch: 101/10000 || Loss: 0.6860083341598511
--------------------------------------------------
Validation || Epoch: 101 || Loss: 0.8921460184183988 || Accuracy: 0.6225854158401489 || F1-score: 0.25579975579975583
Early stopping at epoch 101!
Accuracy on dataset of size 672: 62.35118865966797 %.
Average loss: 0.9267005866224115
proportion of labels in prediction: [tensor(1.), tensor(0.), tensor(0.)]
proportion of labels in data: [tensor(0.6235), tensor(0.2232), tensor(0.1533)]
- f1: [0.76810266 0.         0.        ]
- f1 (average): 0.25603421937060805
- 

## Fine-tuned BERT

### Mean pooled

In [175]:
path_history = obtain_mean_history(pooled_mean, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

NameError: name 'pooled_mean' is not defined

### Max pooled

In [ ]:
path_history = obtain_mean_history(pooled_max, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

### Sum pooled

In [ ]:
path_history = obtain_mean_history(pooled_sum, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

### CLS

In [ ]:
path_history = obtain_mean_history(pooled_cls, path_specifics)
for hidden_dim in hidden_dim_trials:
    print(f"\n********** hidden_dim: {hidden_dim}")
    implement_ffn(x_data=path_history,
                  y_data=y_data,
                  hidden_dim=hidden_dim,
                  learning_rate=learning_rate,
                  loss=loss)

# Baseline 3: LSTM classification

# StackedDeepSigNet

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [188]:
time_features = ["time_encoding", "timeline_index"]
path_specifics = {"pad_by": "history",
                  "zero_padding": True,
                  "method": "k_last",
                  "k": 20,
                  "time_feature": time_features,
                  "standardise_method": ["minmax", None],
                  "embeddings": "dim_reduced",
                  "include_current_embedding": True,
                  "pad_from_below": False}

In [189]:
def obtain_SDSN_input(embeddings, path_specifics):
    reduction = nlpsig.DimReduce(method="gaussian_random_projection", n_components=50)
    embeddings_reduced = reduction.fit_transform(embeddings, random_state=seed)
    
    paths = nlpsig.PrepareData(anno_mi,
                               id_column="transcript_id",
                               label_column="client_talk_type",
                               embeddings=embeddings,
                               embeddings_reduced=embeddings_reduced)
    
    paths.pad(**path_specifics)
    
    paths.array_padded = paths.array_padded[client_index]
    paths.embeddings = paths.embeddings[client_index]
    paths.embeddings_reduced = paths.embeddings_reduced[client_index]
    
    return paths.get_torch_path_for_SDSN(
        include_time_features_in_path=True,
        include_time_features_in_input=True,
        include_embedding_in_input=True,
        reduced_embeddings=False
    )

In [202]:
def implement_sdsn(x_data,
                   y_data,
                   sig_depth,
                   input_channels,
                   output_channels,
                   lstm_hidden_dim,
                   ffn_hidden_dim,
                   BiLSTM,
                   learning_rate,
                   loss,
                   gamma = 0):
    SDSN_args = {
        "input_channels": input_channels,
        "output_channels": output_channels,
        "num_time_features": len(time_features),
        "embedding_dim": x_data.shape[2]-input_channels-len(time_features),
        "sig_depth": sig_depth,
        "hidden_dim_lstm": lstm_hidden_dim,
        "hidden_dim_ffn": ffn_hidden_dim,
        "output_dim": len(label_to_id),
        "dropout_rate": 0.1,
        "augmentation_type": "Conv1d",
        "BiLSTM": BiLSTM,
        "comb_method": "concatenation"
    }
    
    sdsn_model = StackedDeepSigNet(**SDSN_args)
    # print(sdsn_model)
    
    # split dataset
    train, valid, test = split_dataset(x_data=torch.tensor(x_data).float(),
                                       y_data=torch.tensor(y_data),
                                       train_size=0.8,
                                       valid_size=0.5,
                                       shuffle=True,
                                       as_DataLoader=True,
                                       seed=seed)
    
    # define loss
    if loss == "focal":    
        criterion = FocalLoss(gamma = gamma)
    elif loss == "cross_entropy":
        criterion = torch.nn.CrossEntropyLoss()

    # define optimizer
    optimizer = torch.optim.Adam(sdsn_model.parameters(), lr=learning_rate)
    # define scheduler for adjusting the learning rate
    scheduler = ReduceLROnPlateau(optimizer, 'min')
    # scheduler = StepLR(optimizer, step_size = 10, gamma = 0.5)
    # scheduler = CosineAnnealingWarmRestarts(optimizer, 
    #                                         T_0 = 8,# Number of iterations for the first restart
    #                                         T_mult = 1, # A factor increases TiTi after a restart
    #                                         eta_min = learning_rate*0.1)
    # scheduler = None
    
    sdsn_model = training_pytorch(model=sdsn_model,
                                  train_loader=train,
                                  criterion=criterion,
                                  optimizer=optimizer,
                                  num_epochs=10000,
                                  scheduler=scheduler,
                                  valid_loader=valid,
                                  early_stopping=True,
                                  early_stopping_metric="f1",
                                  patience=100,
                                  verbose=True,
                                  verbose_epoch=100,
                                  seed=seed)

    pred, label = testing_pytorch(sdsn_model, test, criterion)
    print(f"proportion of labels in prediction: {[sum(pred==i)/len(pred) for i in label_to_id.values()]}")
    print(f"proportion of labels in data: {[sum(label==i)/len(label) for i in label_to_id.values()]}")
    
    f1_scores = metrics.f1_score(label, pred, average=None)
    print(f"- f1: {f1_scores}")
    print(f"- f1 (average): {sum(f1_scores)/len(f1_scores)}")
    print(f"- accuracy: {sum(pred==label)/len(pred)}")
    
    return sdsn_model

In [203]:
lstm_hidden_dim_trial = [[8,8], [12,12,8]]
ffn_hidden_dim_trial = [[100,100], [100,100,100], [100,100,100,100]]
sig_depth = 3
output_channels = 10
BiLSTM = True

## SBERT 768

In [ ]:
x_data, input_channels = obtain_SDSN_input(sbert_768_embeddings, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.

********** lstm_hidden_dim: [8, 8] || ffnhidden_dim: [100, 100]


/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_74263/2493663396.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train, valid, test = split_dataset(x_data=torch.tensor(x_data).float(),


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1/10000 || Item: 0/85 || Loss: 1.0969207286834717
--------------------------------------------------
##### Epoch: 1/10000 || Loss: 1.0233066082000732
--------------------------------------------------
Validation || Epoch: 1 || Loss: 1.023309436711398 || Accuracy: 0.46359583735466003 || F1-score: 0.3083223136970524


## SBERT 384

In [ ]:
x_data, input_channels = obtain_SDSN_input(sbert_384_embeddings, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

## Pretrained BERT

### Mean pooled

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_mean_pretrained, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

### Max pooled

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_max_pretrained, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

### Sum pooled

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_sum_pretrained, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

### CLS

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_cls_pretrained, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

## Fine-tuned BERT

### Mean pooled

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_mean, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

### Max pooled

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_max, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

### Sum pooled

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_sum, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

### CLS

In [ ]:
x_data, input_channels = obtain_SDSN_input(pooled_cls, path_specifics)
for lstm_hidden_dim in lstm_hidden_dim_trial:
    for ffn_hidden_dim in ffn_hidden_dim_trial:
        print(f"\n********** lstm_hidden_dim: {lstm_hidden_dim} "
              f"|| ffnhidden_dim: {ffn_hidden_dim}")
        implement_sdsn(x_data=x_data,
                       y_data=y_data,
                       sig_depth=sig_depth,
                       input_channels=input_channels,
                       output_channels=output_channels,
                       lstm_hidden_dim=lstm_hidden_dim,
                       ffn_hidden_dim=ffn_hidden_dim,
                       BiLSTM=BiLSTM,
                       learning_rate=learning_rate,
                       loss=loss)

Baselines:
   - just looking at the sentence embeddings (encodes nothing about the history on the post)
       - highlights importance of looking at the sequence
   - averaging history
   - comparing the cosine similarity between previous post and current post to see if switch
   
Test for:
- How many posts do you need to look back?